# Tracking

In [ ]:
!pip install --upgrade opencv-python
!pip install opencv-python numpy


## Importing libraries

In [1]:
# Importing libraries
from PIL import Image
import cv2
from IPython.display import Image, display
import math
import numpy as np
import matplotlib.pyplot as plt 
from shapely.geometry import LineString
from shapely.geometry import Point
from itertools import combinations
import networkx as nx
from shapely.geometry import LineString
import colorsys

## Projecting image to top view

In [2]:
# https://www.geeksforgeeks.org/perspective-transformation-python-opencv/
# https://note.nkmk.me/en/python-opencv-qrcode/
# https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html  
# https://theailearner.com/tag/cv2-minarearect/

def perspective_transformation(image,result):
    height, width, _ = image.shape
    
    # Destination points for the matrix transformation
    #dest_corners =np.float32([(width, height), (0, height), (width, 0), (0, 0)])
    dest_corners = np.float32([(0, height), (width, height), (0, 0), (width, 0)])

    # Mask values of the object to be detected
    (min_blue, min_green, min_red) = (42, 0, 99)
    (max_blue, max_green, max_red) = (85, 255, 255)

    # Initialize a list to store the centers of the detected objects
    centers = []

    # HSV (Hue, Saturation, Value): Separates the color information from the brightness information, making it robust to changes in lighting conditions
    hsv_frame = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    #getting the mask image from the HSV image using threshold values
    mask = cv2.inRange(hsv_frame, (min_blue, min_green, min_red), (max_blue, max_green, max_red))

    #extracting the contours of the object
    contours,_ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    #sorting the contour based of area
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Take the top 4 contours
    top_contours = contours[:4]

    # print('number of contours', len(top_contours))

    # Extract the 4 biggest contours wich are not having the same center
    for contour in top_contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        center = (x + w // 2, y + h // 2)

        # Check if the center is not close to any existing centers
        if all(np.linalg.norm(np.array(center) - np.array(existing_center)) > 50 for existing_center in centers):
            centers.append(center)
            cv2.rectangle(image, (x - 15, y - 15), (x + w + 15, y + h + 15), (0, 255, 0), 4)
    
    # print('number of centers',len(centers))
    #print(centers)
    
    # Display the image with red points for center
    #for center in centers:
    #    cv2.circle(image, tuple(center), 5, (0, 0, 255), -1)

    # Display the image using plt
    #plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    #plt.title('Contours with Rectangles and Red Center Points')
    #plt.axis('on')
    #plt.show()

    if len(centers) == 4:
        center_points = np.float32(centers).reshape(-1, 1, 2)
        transformation_matrix = cv2.getPerspectiveTransform(center_points, dest_corners)
        new_perspective_image = cv2.warpPerspective(result, transformation_matrix, (width, height))
    
    #plt.imshow(cv2.cvtColor(new_perspective_image, cv2.COLOR_BGR2RGB))
    #plt.title('New perspective image')
    #plt.axis('on')
    #plt.show()

        return new_perspective_image, transformation_matrix
    else:
    # Return the initial image if not enough contours are detected
        transformation_matrix = None
        return image, transformation_matrix

In [ ]:
def real_angle(rect, prev_angle):
    angle = rect[2]

    # Check if the rectangle is vertically oriented
    if rect[1][0] < rect[1][1]:
        angle -= 90
    
    
    # Choose the shorter edge to compute the angle between vertical
    width, height = rect[1]
    if width > height:
        angle += 180
    else:
        angle += 90
    
    # Adjust the angle based on previous angle
    num_rotations = int((prev_angle - angle) / 90)
    real_angle = angle - num_rotations * 90
    
    # Convert the angle to the range [0, 360)
    while real_angle < 0:
        real_angle += 360

    while real_angle >= 360:
        real_angle -= 360

    # Update the previous angle for the next iteration
    prev_angle = angle

    return real_angle, prev_angle

In [ ]:
def angle_robot(points, transformation_matrix, prev_angle=0):
    
    if transformation_matrix is not None:        
        # Apply the new perspective on the points
        points = cv2.perspectiveTransform(points.reshape(-1, 1, 2), transformation_matrix)

    # Extract the rotation angle from the rectangle
    # the angle between the line joining the starting and endpoint and the horizontal  [-90,0] 
    rect = cv2.minAreaRect(points)
    angle, prev_angle = real_angle(rect, prev_angle)

    return angle, prev_angle

In [ ]:
# Load the image

# filename = 'Images/green_square_no_QR.png'
filename = 'Images/green_square.png'
image = cv2.imread(filename, cv2.IMREAD_COLOR)
result = image.copy()
height, width, _ = image.shape

# Perform perspective transformation
new_perspective_image = perspective_transformation(image,result)


In [10]:
# https://medium.com/globant/maneuvering-color-mask-into-object-detection-fce61bf891d1

#empty function
def doNothing(x):
    pass

#creating a resizable window named Track Bars
cv2.namedWindow('Track Bars', cv2.WINDOW_NORMAL)

#creating track bars for gathering threshold values of red green and blue
cv2.createTrackbar('min_blue', 'Track Bars', 0, 255, doNothing)
cv2.createTrackbar('min_green', 'Track Bars', 0, 255, doNothing)
cv2.createTrackbar('min_red', 'Track Bars', 0, 255, doNothing)

cv2.createTrackbar('max_blue', 'Track Bars', 0, 255, doNothing)
cv2.createTrackbar('max_green', 'Track Bars', 0, 255, doNothing)
cv2.createTrackbar('max_red', 'Track Bars', 0, 255, doNothing)

resized_image = cv2.resize(image,(800, 626))
#converting into HSV color model
hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)

#showing both resized and hsv image in named windows
cv2.imshow('Base Image', resized_image)
cv2.imshow('HSV Image', hsv_image)


#creating a loop to get the feedback of the changes in trackbars
while True:
    #reading the trackbar values for thresholds
    min_blue = cv2.getTrackbarPos('min_blue', 'Track Bars')
    min_green = cv2.getTrackbarPos('min_green', 'Track Bars')
    min_red = cv2.getTrackbarPos('min_red', 'Track Bars')
    
    max_blue = cv2.getTrackbarPos('max_blue', 'Track Bars')
    max_green = cv2.getTrackbarPos('max_green', 'Track Bars')
    max_red = cv2.getTrackbarPos('max_red', 'Track Bars')
    
    #using inrange function to turn on the image pixels where object threshold is matched
    mask = cv2.inRange(hsv_image, (min_blue, min_green, min_red), (max_blue, max_green, max_red))
    #showing the mask image
    cv2.imshow('Mask Image', mask)
    # checking if q key is pressed to break out of loop
    key = cv2.waitKey(25)
    if key == ord('q'):
        break
        
#printing the threshold values for usage in detection application
print(f'min_blue {min_blue}  min_green {min_green} min_red {min_red}')
print(f'max_blue {max_blue}  max_green {max_green} max_red {max_red}')
#destroying all windows
cv2.destroyAllWindows()


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2561: error: (-27:Null pointer) NULL window: 'Track Bars' in function 'cvGetTrackbarPos'


## Start Tracking of QR code position and angle in the new perspective


In [9]:
window_name = 'Tracking QR Code'
camera_id = 0 # Try changing with one of these (0,1,2..) because it depends how amny cameras are connected to your PC

video_stream = cv2.VideoCapture(camera_id)
QR_detector = cv2.QRCodeDetector()
prev_angle = 1

# Check if the webcam is opened correctly
if not video_stream.isOpened():
    raise IOError("Cannot open webcam")

while True:
    # Capture a frame from the video stream
    ret, image = video_stream.read()
    if ret:
        # Detect QR codes in the captured frame
        ret_qr_code, points, _ = QR_detector.detectAndDecode(image)
        if ret_qr_code:
            img_copy = image.copy()
            
            color = (0, 255, 0)
            img_copy = cv2.polylines(img_copy, [points.astype(int)], isClosed=True, color=color, thickness=8)
            
            # Apply the new percpective on the frame
            new_perspective_image, transformation_matrix = perspective_transformation(image,img_copy)
                        
            #angle, prev_angle = angle_robot(points, transformation_matrix, prev_angle)
            #print(angle)
            
            # Display the modified image in the window
            cv2.imshow(window_name, new_perspective_image)
            # Display the initial image in the window
            #cv2.imshow(window_name, img_copy)

    # Exiting the loop when the 'Esc' key is pressed
    c = cv2.waitKey(1)
    if c == 27:
        break

video_stream.release()
cv2.destroyAllWindows()